In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [10]:
df = pd.read_csv('DatasetCredit-g.csv')

# Display the first few rows
#print(df.head())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   int64  
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   int64  
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   int64  
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   int64  
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

In [ ]:

data_frame = pd.DataFrame(df, columns=["class","foreign_worker"])
print(data_frame)

In [3]:


# Example DataFrame
#data_frame = pd.DataFrame({'Category': ['cinema', 'education', 'cinema', 'education']})

# Define mapping
mapping_foreign = {'no': 0, 'yes': 1}
mapping = {'bad': 0, 'good': 1}

# Apply mapping
df['class_binary'] = df['class'].map(mapping)
df['foreign'] = df['foreign_worker'].map(mapping_foreign)
df.drop(columns=['foreign_worker', 'class'])
print(df)

    checking_status  duration                  credit_history  \
0                <0         6  critical/other existing credit   
1          0<=X<200        48                   existing paid   
2       no checking        12  critical/other existing credit   
3                <0        42                   existing paid   
4                <0        24              delayed previously   
..              ...       ...                             ...   
995     no checking        12                   existing paid   
996              <0        30                   existing paid   
997     no checking        12                   existing paid   
998              <0        45                   existing paid   
999        0<=X<200        45  critical/other existing credit   

                 purpose  credit_amount    savings_status  employment  \
0               radio/tv         1169.0  no known savings         >=7   
1               radio/tv         5951.0              <100      1<=X<4   


In [ ]:
// df_binary = pd.get_dummies(data_frame["class","foreign_worker"], prefix='Binary')
// print(df_binary)


In [ ]:
// Isto preve o risco por agora, dps usar  amesma logica ma para credito, ja temos de criar outro modelo
// Com o teste, verifica a previsao e verifica overfitting e isso

In [8]:

data_x = df.drop(columns=["class_binary"])
X= data_x
y= df["class_binary"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

print(classification_report(y_pred, y_test)) 



              precision    recall  f1-score   support

           0       1.00      1.00      1.00        91
           1       1.00      1.00      1.00       209

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [ ]:
param_grid_search = {
    'n_estimators': [100,200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid = param_grid_search. cv=5)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Estimator:", grid_search.best_estimator_)

In [ ]:
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   param_grid)
random_search.fit(X_train, y_train)
print(random_search.best_estimator_) // este random search demora mais mas costuma dar melhor resultados, que o problema do gridsearch é que sao valores já que nos definimos

In [ ]:
#Aqui o P é o valor previsto do nosso modelo, o Y é o suposto "desejado", não sei bem o que quer dizer
#tenho que fazer dois modelos, um para prever o risco e outro para prever o credit amount requested mesmo para utilizar na função como o P

def calculate_f(Y, P):
    if abs(Y - P) / Y < 0.3:
        return 100
    else:
        return 0


In [ ]:
# Test the function with different Y and P values
Y = 100
P = 70

result = calculate_f(Y, P)
print(f"f({Y}, {P}) = {result}")


In [ ]:
# Utilizar XGBoost regression como usaste em DAA, ver como fizeste isso em DAA

In [ ]:
# falta me só ver como é que vou passar as labels